[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Humboldt-WI/adams/blob/master/exercises/Ex08_forecasting_financial/Ex08a_LSTM_peculiarities.ipynb) 

# LSTM peculiarities

Having looked into the computations within a RNN and a LSTM cell by writing numpy code, the previous tutorial has equipped us with an understanding of recurrent computations. Our next task is to move on to Keras and build proper RNNs - we will focus on LSTMs - in that framework. It turns out that this task is not quite as easy as when defining and training a feedforward network. Sequence data involves some additional challenges. The implementation of the LSTM in Keras also exhibits some peculiarities that we have to be aware of. 

The purpose of this short tutorial is to review some of the new challenges mentioned above and to provide some introductory content to help get you started. We will only briefly touch on selected concepts and revisit these in the next tutorial on financial time series forecasting using an LSTM.

## LSTM  peculiarities #1: input/output

LSTMs are flexible in terms of the structure of the input and the output sequence. This picture from A. Karpathy's blog reminds us of the possibilities. 

<img src="RNN_structures.jpg" style="width: 800px;"/>
Source: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

Applications like:
1. Image recognition (binary outcome like cat/dog) - strictly speaking its not a sequence
2. Picture description (outcome "Tall man in red shirt holds beer) - not really for Keras
3. Sentiment analysis, typeing suggestions or our time series prediction (using a bunch of past values to predict one in the future)
4. Translation or also can be prediction of several steps in time series
5. May be used for example for ongoing video classification

Additionally we have an architecture called **Bidirectional LSTM** that not only preserves information from the past and passes it to the future but also passes information from the future to the past (used widely for the tasks where context matters a lot).

## LSTM  peculiarities #2: train set structure
In the manual implementation, we have touched on how the data can be structured before feeding it into the model. Ultimately, we always want to have a set of features and a target. Although LSTMs are designed as models for sequential data, it turns out that they do not work well if the length of the input sequence is too long. As a rule of thumb, many sources mention a range of 200 - 400 time steps, which an LSTM can still process. When saying "can", you want to note that the quality of a predictive model can always be evaluated by measuring forecast accuracy. In addition to decreased accuracy, issues with too long sequences also concern the speed of training and the vanishing gradient problem. It is common practice to split a long sequence into subsequences and using the resulting "chunks of data" as input for the LSTM. This is the approach we adopted in the last tutorial. Remember this picture:

<img src="time_series_one_step.png" style="width: 500px;"/>

An important questions arises: what should be the length of the subsequence or time window? It will depend a lot on field knowledge. For example, in the case of sentiment analysis, the question is how many words you need on average to convey a sentiment. With ordinary time series, you can examine auto-correlation or partial auto-correlation to derive a judgment.

Next, relating the previous picture to the picture on possible LSTM setups, we note that we could also use a LSTM to forecast multiple steps ahead and not just one. This would look as follows:  

<img src="time_series_sequence.png" style="width: 500px;"/>

We decide on the sequence length and keep it constant. Our input and output sequences are now of the same length. Each new sequence starts one time step ahead of the sample sequence. The remaining values, i.e. the remainder of the full sequence length divided by the chosen length, are dropped. Keep in mind that then your model turns into seq2seq type. By making the network stateful, we use the hidden state at the end of the previous sequence as the starting point of the next sequence. Since the hidden state summarizes information from earlier observation, this increases the information available for each prediction beyond the window size. 

Last, note that a sequence learning task could also have an entirely different form of time series or text data. To illustrate this, let's say we are expecting the model to figure out that it has to count the number of ones in an input sequence and give it as prediction.

In [ ]:
     X       y
[0,1,1,1,2] [3]
[0,0,1,1,2] [2]
[0,0,0,1,2] [1]
[1,0,0,1,2] [2]
[0,0,0,0,1] [0]
...

In this case, the order of those observations does not matter, although it is a sequence problem. We could shuffle the observations as we often do when working with non-sequential data, hoping that shuffling gives better generalization.

In any case, remember that Keras expects the input data for an LSTM to be a 3d tensor with dimensions:
- Number of samples or batch size
- Number of time steps
- Number of features

## LSTM  peculiarities #3: state
Keras facilitates two types of LSTMs: **stateful** or **stateless**

These terms are somewhat confusing. Isn't the very idea of RNNs and latent variable models to keep a hidden state in which information from processing past time steps accumulates? Of course it is. In Keras, the difference between a stateful and a stateless LSTM concerns the point when the hidden state is reset. The maybe more common case is a stateless LSTM. There, the hidden state is reset after processing one batch (see below for peculiarities wrt batches). 


Recall that longer time series are commonly split into chunks. This resetting approach enforces what we said above. The time window or length of a subsequence in which a longer time series is split is crucial. A *stateless* LSTM will not maintain state information across multiple batches. The information that you deem relevant to forecast must be included in the subsequence. 

*Stateful* LSTMs are Keras's solution toward maintaining state when working with longer sequences. Conceptually, *stateful* LSTM are conceptually more suitable for such settings. The splitting of the time series into chunks is somewhat arbitrary and done because not doing it would probably break LSTM training. You would not bother with splitting a time series into chunks when using an econometric model like ARIMA.   

Before demonstrating *stateful* and *stateless* LSTMs in the next tutorial, let's note that a stateful recurrent model is one for which the internal states (memories) obtained after processing a batch of samples are reused as initial states for the samples of the next batch. This allows processing longer sequences while keeping computational complexity manageable. Again, the hidden state is passed from batch to batch, not within the batch. Within a batch the sub-sequences are treated as independent.

If the model is stateless, the cell states are reset after each batch. It is considered to be more efficient in implementations than stateful and the clear choice when every observation does not depend on the previous one (e.g., sentence classification). Additionally, stateless LSTMs can be implemented with and without shuffling of observations.


This [blog post](http://philipperemy.github.io/keras-stateful-lstm/) offers a very good explanation of stateful versus stateless and why the two are distinguished at all. It also offers a nice demo. Same more demos are available in [this tutorial](https://machinelearningmastery.com/stateful-stateless-lstm-time-series-forecasting-python/).

## LSTM  peculiarities #4: batches 

When it comes to time series forecasting, there are many suggestions what should be the batch size for an LSTM. One of the rules one can observe is your batch size is the same as your output (according to Keras functionality). In case your output is next day prediction, you might want to consider batch size=1. More specifically, the Keras implementation enforces a constant batch size for training and prediction (see, e.g., [here for a discussion](https://stackoverflow.com/questions/43702481/why-does-keras-lstm-batch-size-used-for-prediction-have-to-be-the-same-as-fittin)). Thus, if you want to make prediction one day into the future, a common setting in stock prediction settings, then you should train your LSTM with batch size equal to one. Conversely, if you train your LSTM with batch size equal to, say, 10, you must provide exactly ten samples to the *.predict()* function. In practice, the latter implies that you would need to wait for ten days before you can make a prediction. I know this sounds very serious. Relax, there are ways to get round this issue. One is to use Tensorflow but you can [also *persuade* Keras to let you change the batch size from training to prediction].(https://machinelearningmastery.com/use-different-batch-sizes-training-predicting-python-keras/)  

In general, the advice is to keep batch size a value that divides without remainder into the train and validation set sizes, so that no data gets discarded. Keep in mind that the LSTM will be chaining together line 1 of batch n with line 1 of batch n+1 (see below). So try to keep the batch size somewhat meaningful. 

However, when it comes to text processing, the situation changes. [Jeremy Howard](https://www.youtube.com/watch?v=H3g26EVADgY&feature=youtu.be&t=17m50sin) advocates that splitting a big string of text into chunks, stacking it and then creating batches from the first "slice", second "slice" and etc. offers nice parallelization capacities and does not really harm the training process. Keep in mind that in case you have short sequences (that are still longer than your window size) this method might bring in additional distortion.

Example (mind that numbers are only used for simplicity, should rather be words: we have a "long" string [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17......1000]
Let's say we want to have batch=10, then we split the string into 10 chunks (that would make 100 numbers per chunk) and stack them:

[1.....100]

[101...200]

[201...300]

...

[900...1000]

(10 rows, 100 columns)

Now, if you remember in order to transform it into a supervised task we need to decide on the "window" size and the output type. Let's say we will go for a many-to-many architecture, so we choose the window=10 and same size output without overlap. 
Then our stack would look like this:

[1..10][11...20][21...30]......[91..100] 

[101..110][111...120][121...130]......[191..200]

.........

[901..910][911...920][921...930]......[991..1000]

That is [1..10] will be used to predict [2...11], [11...20] will be used to predict [12...21] and so forth.


Then, the first batch would be the first slice/column:

[1..10]

[101..110]

....

[901..910]


Second batch - second column and etc. where we use a stateful network to continue with the hidden state from the previous batch.

Structuring the data in this way, the sequence is lost at the end points of each row (i.e. 100 to 101, 200 to 201). If we consider the sequence to be not hundreds but thousands of words, that may be something we are willing to sacrifice, because we gain a parallelization of the process: the 10 chunks/rows are training simultaneously with every batch. 

## LSTM  peculiarities #5: Resetting ###
In case you are using a stateful LSTM, make sure you reset the state after every epoch, otherwise the NN will treat it as a continuation of the time series.


In [ ]:
epochs=30

for i in range(epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
        model.reset_states()
        
# Also you might want to consider the online forecast
for i in range(len(X)):
        testX, testy = X[i], y[i]
        testX = testX.reshape(1, 1, 1)
        yhat = model.predict(testX, batch_size=1)
        print('>Expected=%.1f, Predicted=%.1f' % (testy, yhat))